In [4]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
import os

/Users/amber/Documents/Learnings_2024/Langraph_2025/langraph_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load environment variables from the .env file
load_dotenv()

True

In [6]:

# # Access the variables
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [7]:
# using langchain_google_genai
gemini_model = ChatGoogleGenerativeAI(model='gemini-1.5-flash',google_api_key=gemini_api_key)
result = gemini_model.invoke('What is the capital of India')

print(result.content)

The capital of India is **New Delhi**.


In [8]:

# create a state

class LLMState(TypedDict):

    question: str
    answer: str

In [12]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = gemini_model.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [13]:
# create our graph

graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [14]:
# execute

intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state['answer'])

The distance between the Earth and the Moon varies because the Moon's orbit is not a perfect circle.  The average distance is about **238,855 miles (384,400 kilometers)**.


In [15]:

gemini_model.invoke('How far is moon from the earth?').content

"The distance between the Earth and the Moon is not constant because the Moon's orbit is elliptical.  The average distance is about **238,855 miles (384,400 kilometers)**."